#TASK 6: DIY
###Build a NN as they did in Stone Age.

Although the name of the task needs no explanation, the purpose of this task is to create a NN from scratch, with no help from keras, tensorflow or torch. How are we going to be able to do that, in these modern times?  
Our only hope is to follow: https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

Please, let us mount your google drive.

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


First of all, we need the dataset. As proposed on the link, let's download the dataset.

In [2]:
def data_downloader():
    print('\n ** Checking dataset files...')
    if os.path.exists('./wheat-seeds.csv'):
        print(" ** Files from dataset were already downloaded.")
    else:
        print("n ** Downloading wheat-seeds.csv...")
        !wget 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv'
        print(" ** Download successful.")

import os

%cd "/content/"
data_downloader()
!ls

/content

 ** Checking dataset files...
 ** Files from dataset were already downloaded.
drive  sample_data  wheat-seeds.csv


Now, let's take a closer look into this dataset.

In [3]:
import csv
from csv import reader
import pandas as pd

data = pd.read_csv("wheat-seeds.csv", header=None)
print(" ** Reading it with pandas, we may use .describe() function to gather some information.")
data.describe()

 ** Reading it with pandas, we may use .describe() function to gather some information.


,0,1,2,3,4,5,6,7
count,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000
mean,14.847524,14.559286,0.870999,5.628533,3.258605,3.700201,5.408071,2.000000
std,2.909699,1.305959,0.023629,0.443063,0.377714,1.503557,0.491480,0.818448
min,10.590000,12.410000,0.808100,4.899000,2.630000,0.765100,4.519000,1.000000
25%,12.270000,13.450000,0.856900,5.262250,2.944000,2.561500,5.045000,1.000000
50%,14.355000,14.320000,0.873450,5.523500,3.237000,3.599000,5.223000,2.000000
75%,17.305000,15.715000,0.887775,5.979750,3.561750,4.768750,5.877000,3.000000
max,21.180000,17.250000,0.918300,6.675000,4.033000,8.456000,6.550000,3.000000


Fortunately, this a beautiful dataset with no missing data, class balanced. Still, it's worth a check:

In [4]:
medians = dict()
for i in range(int(len(data.columns))):
    medians[i] = data.iloc[:,i].median()
    data.iloc[:,i] = data.iloc[:,i].replace(to_replace=0, value=medians[i])

Now, we may proceed. Let's convert those data to arrays.

In [5]:
import numpy as np
print(' ** Converting pandas dataframe to np.arrays of x_data (inputs) \
and y_data (labels).')
x_data = data.iloc[:,:7].values
y_data = data.iloc[:,7:8].values

print(' ** x_data: %s, y_data: %s.' % (x_data.shape, y_data.shape))

print(" ** We're gonna reload those images to keep them in accord to the program.")

def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

def str_column_to_int(dataset, column):
	  class_values = [row[column] for row in dataset]
	  unique = set(class_values)
	  lookup = dict()
	  for i, value in enumerate(unique):
		    lookup[value] = i
	  for row in dataset:
		    row[column] = lookup[row[column]]
	  return lookup

def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

print(" ** Converting y_data to integer values.")
for i in range(len(y_data[0])-1):
  	str_column_to_float(y_data, i)
   
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats
 
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

 ** Converting pandas dataframe to np.arrays of x_data (inputs) and y_data (labels).
 ** x_data: (210, 7), y_data: (210, 1).
 ** We're gonna reload those images to keep them in accord to the program.
 ** Converting y_data to integer values.


It it convenient to separate a train and test set. Let's do that. Knowing the dataset consists of 210 rows, let's separate 20% for testing.

In [6]:
test_frac = int(len(y_data)*0.2)
y_size = int(len(y_data))
lim = y_size - test_frac

x_test = x_data[lim:y_size]
y_test = y_data[lim:y_size]

x_train = x_data[0:lim]
y_train = y_data[0:lim]

print(' ** x_train: %s, y_train: %s.' % (x_train.shape, y_train.shape))
print(' ** x_test: %s, y_test: %s.' % (x_test.shape, y_test.shape))

 ** x_train: (168, 7), y_train: (168, 1).
 ** x_test: (42, 7), y_test: (42, 1).


Finally, let's begin the construction of our neural network. Let's build functions that defines random weights for the hidden_layer and output_layer.

In [7]:
def initialize_network(n_inputs, n_hidden, n_outputs):
	  network = list()
	  hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	  network.append(hidden_layer)
	  output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	  network.append(output_layer)
	  return network

Python already has in-built functions which gives "random numbers", from random library. Actually, it's a list of numbers located in a given seed. It is possible to choose between different seeds if necessary.

In [8]:
from random import seed
from random import random
from random import randrange
from math import exp

seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
	  print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


This is how we can manually calculate a neuron activation for an input.

In [9]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	  activation = weights[-1]
	  for i in range(len(weights)-1):
		    activation += weights[i] * inputs[i]
	  return activation

Now let's make an activation function.

In [10]:
def transfer(activation):
	  return 1.0 / (1.0 + exp(-activation))

It will be necessary to work on backpropagation as well. It will backpropagate an input to the network output.

In [11]:
def forward_propagate(network, row):
	  inputs = row
	  for layer in network:
		    new_inputs = []
		    for neuron in layer:
			      activation = activate(neuron['weights'], inputs)
			      neuron['output'] = transfer(activation)
			      new_inputs.append(neuron['output'])
		    inputs = new_inputs
	  return inputs

This function will calculate the derivative of the output from a neuron.

In [12]:
def transfer_derivative(output):
	  return output * (1.0 - output)

Here comes the error backpropagation function, and storage of it in inputs.

In [13]:
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

Now is the weights update, according to the error.

In [14]:
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

Finally, after defining all those necessary steps on a neural network, we may define the training function.

In [15]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	  for epoch in range(n_epoch):
		    sum_error = 0
		    for row in train:
			      outputs = forward_propagate(network, row)
			      expected = [0 for i in range(n_outputs)]
			      expected[row[-1]] = 1
			      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			      backward_propagate_error(network, expected)
			      update_weights(network, row, l_rate)
		    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))


It is necessary to build a function for prediction as well.

In [16]:
def predict(network, row):
	  outputs = forward_propagate(network, row)
	  return outputs.index(max(outputs))

Now, let's define a function to separate dataset into folds for cross validation.

In [17]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

Let's define some metric functions in order to evaluate our model.

In [18]:
def accuracy_metric(actual, predicted):
	  correct = 0
	  for i in range(len(actual)):
		    if actual[i] == predicted[i]:
			      correct += 1
	  return correct / float(len(actual)) * 100.0

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

Last but not least, Stochastic Gradient Descent could be useful for backpropagation.

In [19]:
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	  n_inputs = len(train[0]) - 1
	  n_outputs = len(set([row[-1] for row in train]))
	  network = initialize_network(n_inputs, n_hidden, n_outputs)
	  train_network(network, train, l_rate, n_epoch, n_outputs)
	  predictions = list()
	  for row in test:
		    prediction = predict(network, row)
		    predictions.append(prediction)
	  return (predictions)

Finally, it's time for some action! Let's see it running!

In [20]:
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
n_outputs = 3

dataset = load_csv('wheat-seeds.csv')
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i) 
str_column_to_int(dataset, len(dataset[0])-1)

minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

>epoch=0, lrate=0.300, error=129.025
>epoch=1, lrate=0.300, error=106.656
>epoch=2, lrate=0.300, error=96.885
>epoch=3, lrate=0.300, error=85.680
>epoch=4, lrate=0.300, error=76.587
>epoch=5, lrate=0.300, error=70.338
>epoch=6, lrate=0.300, error=66.016
>epoch=7, lrate=0.300, error=62.689
>epoch=8, lrate=0.300, error=59.725
>epoch=9, lrate=0.300, error=56.740
>epoch=10, lrate=0.300, error=53.515
>epoch=11, lrate=0.300, error=49.994
>epoch=12, lrate=0.300, error=46.316
>epoch=13, lrate=0.300, error=42.744
>epoch=14, lrate=0.300, error=39.486
>epoch=15, lrate=0.300, error=36.632
>epoch=16, lrate=0.300, error=34.187
>epoch=17, lrate=0.300, error=32.113
>epoch=18, lrate=0.300, error=30.354
>epoch=19, lrate=0.300, error=28.856
>epoch=20, lrate=0.300, error=27.569
>epoch=21, lrate=0.300, error=26.456
>epoch=22, lrate=0.300, error=25.484
>epoch=23, lrate=0.300, error=24.630
>epoch=24, lrate=0.300, error=23.874
>epoch=25, lrate=0.300, error=23.201
>epoch=26, lrate=0.300, error=22.599
>epoch=27

And here comes the scores!

In [21]:
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [95.23809523809523, 100.0, 95.23809523809523, 90.47619047619048, 100.0]
Mean Accuracy: 96.190%
